In [9]:
import numpy as np
import cv2
import sys

In [10]:
def construct_yolo_v3():
    path = './yolo_dataSet/'

    # COCO 데이터셋 클래스 이름 가져오기
    with open(path + 'coco_names.txt', 'r') as f:
        class_names = [line.strip() for line in f.readlines()]

    # YOLOv3 모델과 가중치 로드
    model = cv2.dnn.readNet(path + 'yolov3.weights', path + 'yolov3.cfg')

    # 모든 레이어의 이름 가져오기
    layer_names = model.getLayerNames()

    # 출력 레이어 결정
    out_layers_indices = model.getUnconnectedOutLayers()

    # 출력 레이어 이름 리스트 구성
    out_layers = [layer_names[idx - 1] for idx in out_layers_indices]

    return model, out_layers, class_names

In [11]:
def yolo_detect(img, yolo_model, out_layers):
    # 이미지의 크기 정보
    height, width = img.shape[:2]

    # 이미지에서 네트워크에 입력할 blob을 생성(swapRB는 오픈cv BGR이라서 RGB로 바꿔주는거)
    ## 데이터 전처리 작업
    test_img = cv2.dnn.blobFromImage(img, 1.0 / 255, (448, 448), (0,0,0), swapRB=True)

    # YOLO 모델에 blob을 입력으로 설정
    yolo_model.setInput(test_img)

    # 네트워크 실행 및 출력 레이어에서 결과 가져오기
    outputs = yolo_model.forward(out_layers)

    # 검출된 박스, 신뢰도, 각 클래스 id
    box, conf, id = [], [], []

    # 반복문을 사용하여 Box 생성
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > .5: # 신뢰도 50%이상인 경우만 처리
                # 중심점 찾기
                center_x, center_y = int(detection[0] * width), int(detection[1] * height)
                # 가로 세로 폭 찾기
                w, h = int(detection[2] * width), int(detection[3] * height)
                # 시작점 찾기
                x, y = int(center_x - w / 2), int(center_y - h / 2)
                box.append([x, y, x+w, y+h])
                conf.append(float(confidence))
                id.append(class_id)

    # NMS를 사용하여 중복된 박스를 제거한다.
    indices = cv2.dnn.NMSBoxes(box, conf, .3, .6)
    objects = [box[i] + [conf[i]] + [id[i]] for i in range(len(box)) if i in indices]

    return objects

In [12]:
# 모델 생성
model, out_layers, class_names = construct_yolo_v3()

# 클래스별로 색상 지정
colors = np.random.uniform(0, 255, size=(len(class_names), 3))

# 이미지 불러오기
path = './yolo_dataSet/'
img = cv2.imread(path + 'camera.jpg')
if img is None:
    sys.exit('파일 없음')

# Yolo Detecting
res = yolo_detect(img, model, out_layers)

for i in range(len(res)):
    x1, y1, x2, y2, confidence, id = res[i]
    text = f"{class_names[id]}: {confidence: .3f}"
    cv2.rectangle(img, (x1, y1), (x2, y2), colors[id], 2)
    cv2.putText(img, text, (x1, y1+30), cv2.FONT_HERSHEY_PLAIN, 1.5, colors[id])

# 결과 보기
cv2.imshow("YOLO V3_Object Detection", img)
cv2.waitKey()
cv2.destroyAllWindows()

영상 박스 체크

In [86]:
pip install pytube

   ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
   -------------- ------------------------- 20.5/57.6 kB 640.0 kB/s eta 0:00:01
   ---------------------------------------- 57.6/57.6 kB 1.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [13]:
from pytube import YouTube
import numpy as np
import cv2
import sys

# 유튜브 영상 다운로드
def download_youtube_video(url, save_path='./youtube.mp4'):
    yt = YouTube(url)
    stream = yt.streams.filter(file_extension = 'mp4').first()
    stream.download(filename = save_path)
    print(f"영상이 다운로드 되었습니다. 위치({save_path})")

# 유튜브 url
yt_url = 'https://www.youtube.com/watch?v=ARF-GBb0DxU'

download_youtube_video(yt_url)

영상이 다운로드 되었습니다. 위치(./youtube.mp4)


In [14]:
def construct_yolo_v3():
    path = './yolo_dataSet/'

    # COCO 데이터셋 클래스 이름 가져오기
    with open(path + 'coco_names.txt', 'r') as f:
        class_names = [line.strip() for line in f.readlines()]

    # YOLOv3 모델과 가중치 로드
    model = cv2.dnn.readNet(path + 'yolov3.weights', path + 'yolov3.cfg')

    # 모든 레이어의 이름 가져오기
    layer_names = model.getLayerNames()

    # 출력 레이어 결정
    out_layers_indices = model.getUnconnectedOutLayers()

    # 출력 레이어 이름 리스트 구성
    out_layers = [layer_names[idx - 1] for idx in out_layers_indices]

    return model, out_layers, class_names

In [15]:
def yolo_detect(img, yolo_model, out_layers):
    # 이미지의 크기 정보
    height, width = img.shape[:2]

    # 이미지에서 네트워크에 입력할 blob을 생성(swapRB는 오픈cv BGR이라서 RGB로 바꿔주는거)
    ## 데이터 전처리 작업
    test_img = cv2.dnn.blobFromImage(img, 1.0 / 255, (448, 448), (0,0,0), swapRB=True)

    # YOLO 모델에 blob을 입력으로 설정
    yolo_model.setInput(test_img)

    # 네트워크 실행 및 출력 레이어에서 결과 가져오기
    outputs = yolo_model.forward(out_layers)

    # 검출된 박스, 신뢰도, 각 클래스 id
    box, conf, id = [], [], []

    # 반복문을 사용하여 Box 생성
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > .5: # 신뢰도 50%이상인 경우만 처리
                # 중심점 찾기
                center_x, center_y = int(detection[0] * width), int(detection[1] * height)
                # 가로 세로 폭 찾기
                w, h = int(detection[2] * width), int(detection[3] * height)
                # 시작점 찾기
                x, y = int(center_x - w / 2), int(center_y - h / 2)
                box.append([x, y, x+w, y+h])
                conf.append(float(confidence))
                id.append(class_id)

    # NMS를 사용하여 중복된 박스를 제거한다.
    indices = cv2.dnn.NMSBoxes(box, conf, .3, .6)
    objects = [box[i] + [conf[i]] + [id[i]] for i in range(len(box)) if i in indices]

    return objects

In [16]:
cap = cv2.VideoCapture('youtube.mp4')

frame_w = int(cap.get(3))
frame_h = int(cap.get(4))

out = cv2.VideoWriter('detected.mp4', cv2.VideoWriter_fourcc(*'XVID'), 10.0, (frame_w, frame_h))

model, out_layers, class_names = construct_yolo_v3()
colors = np.random.uniform(0, 255, size=(len(class_names), 3))

In [17]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    res = yolo_detect(frame, model, out_layers)

    for i in range(len(res)):
        x1, y1, x2, y2, confidence, id = res[i]
        text = f"{class_names[id]}: {confidence: .3f}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), colors[id], 2)
        cv2.putText(frame, text, (x1, y1+30), cv2.FONT_HERSHEY_PLAIN, 1.5, colors[id])
    out.write(frame)

    cv2.imshow('Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

웹캠

In [ ]:
# Yolo 모델 불러오기
model, out_layers, class_names = construct_yolo_v3()

# 클래스 별로 색상 지정
colors = np.random.uniform(0, 255, size=(len(class_names), 3))

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
if not cap.isOpened():
    sys.exit('카메라 연결 실패')

while True:
    ret, frame = cap.read()
    if not ret:
        sys.exit('프레임 획득 실패')

    res = yolo_detect(frame, model, out_layers)

    for i in range(len(res)):
        x1, y1, x2, y2, confidence, id = res[i]
        text = f"{class_names[id]}: {confidence: .3f}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), colors[id], 2)
        cv2.putText(frame, text, (x1, y1+30), cv2.FONT_HERSHEY_PLAIN, 1.5, colors[id])
    out.write(frame)

    cv2.imshow('Detection', frame)

    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()